In [18]:
%load_ext jupyternotify

import pandas as pd
import numpy as np
import statsapi

from tqdm.auto import tqdm
import datetime
from datetime import date
# from pydrive2.auth import GoogleAuth
# from pydrive2.drive import GoogleDrive

<IPython.core.display.Javascript object>

In [3]:
np.set_printoptions(suppress=True)

In [1]:
# from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
# # Get current time
# current_time = pd.to_datetime(datetime.datetime.now())
# today = pd.to_datetime(date.today())
# print(f"Login time: {current_time}") 

Login time: 2022-09-06 12:46:29.672931


In [4]:
# # Google drive authentication
# auth_ok = False
# try:
#     gauth = GoogleAuth(settings_file='settings.yaml') # configure Auth with settings file
#     if not gauth.credentials:
#         gauth.GetFlow()
#         gauth.flow.params.update({'access_type': 'offline'})
#         gauth.flow.params.update({'approval_prompt': 'force'})
#         gauth.LocalWebserverAuth() # manual authentication if Auth didn't work
#     elif gauth.access_token_expired: # refresh access token if expired
#         try:
#             gauth.Refresh()
#         except Exception as e:
#             gauth.Authorize()
#     else: # auto Auth working
#         gauth.Authorize()
#     gauth.SaveCredentialsFile("mycreds.txt") # save credentials

#     drive = GoogleDrive(gauth)
#     auth_ok = True
#     print("    Google authentication successful.")
# except:
#     print("    Google authentication failed. New data will not be uploaded to google drive.")

In [5]:
def pull_api(game_id): 
    '''Pull MLB game data (game_type, game_num, date, teams, temperature, wind, etc.) from mlb_stats_api. '''
    data = statsapi.get('game', {'gamePk': game_id}, force=False)

    game_pk = game_id
    try:
        game_type = data['gameData']['game']['type']
    except:
        game_type = np.nan

    try:
        game_num = data['gameData']['game']['gameNumber']
    except:
        game_num = np.nan

    try:
        officialDate = data['gameData']['datetime']['officialDate']
    except:
        officialDate = np.nan

    try:
        away_team = data['gameData']['teams']['away']['teamCode']
    except:
        away_team = np.nan

    try:
        away_team_abb = data['gameData']['teams']['away']['abbreviation']
    except:
        away_team_abb = np.nan

    try:
        home_team = data['gameData']['teams']['home']['teamCode']
    except:
        home_team = np.nan

    try:
        home_team_abb = data['gameData']['teams']['home']['abbreviation']
    except:
        home_team_abb = np.nan

    try:
        weather_cond = data['gameData']['weather']['condition']
    except:
        weather_cond = np.nan

    try:
        temp = data['gameData']['weather']['temp']
    except:
        temp = np.nan

    try:
        wind = data['gameData']['weather']['wind']
    except:
        wind = np.nan

    try:
        ump_id = data['liveData']['boxscore']['officials'][0]['official']['id']
    except:
        ump_id = np.nan

    try:
        ump_name = data['liveData']['boxscore']['officials'][0]['official']['fullName']
    except:
        ump_name = np.nan

    return np.array(
        [game_pk, game_num, game_type, officialDate, away_team, away_team_abb, home_team, home_team_abb, weather_cond,
         temp, wind, ump_id, ump_name])

In [6]:
df_all = pd.read_csv('data/PitchData_15-22.csv', index_col=[0])

/Users/honeybunny/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3135: DtypeWarning: Columns (44) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
new_game_ids = df_all[df_all['game_type']=='R']['game_pk'].unique()

In [11]:
new_game_ids

array([663451, 663352, 663272, ..., 413650, 413649, 413661])

In [20]:
len(new_game_ids)

17904

In [32]:
# split full list into 7 chunks with 500000 elements each
chunk_size = 3000
full_list = new_game_ids
chunked_list = [full_list[i:i+chunk_size] for i in range(0, len(full_list), chunk_size)]

In [33]:
def get_data(new_game_ids):
    new_game_arr = np.empty([0, 13])
    for game_id in tqdm(new_game_ids):
        new_game_arr = np.concatenate([new_game_arr, [pull_api(game_id)]])

    new_game_df = pd.DataFrame(data=new_game_arr, columns=['game_pk', 'game_num', 'game_type', 'official_date',
                                                           'away_team', 'away_team_abb', 'home_team', 'home_team_abb',
                                                           'weather_cond', 'temp', 'wind', 'ump_id', 'ump_name'])
    
    return new_game_df

In [35]:
df_list = []
for chunk in tqdm(chunked_list):
    df = get_data(chunk)
    df_list.append(df)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/2904 [00:00<?, ?it/s]

In [39]:
df_all = pd.DataFrame()
for df in df_list:
    df_all = pd.concat([df_all, df])

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
df_all[df_all['wind'].isna()]

df_all.loc[2054, 'wind'] = '0 mph, L To R'

wind = df_all['wind'].str.split(',', expand=True).copy()
wind[['speed', 'unit']] = wind[0].str.split(' ', expand=True).copy()
wind['speed'] = wind['speed'].astype(int)
wind[1] = wind[1].str.strip()

df_all[['wind_speed', 'wind_direction']] = wind[['speed', 1]]

In [41]:
# df_all.to_csv('GameData_15-22.csv')